# Homework Week 8

For each of the following questions please show that the statistical assumptions for each of the respective tests are met. It is good practice always before start using statistical analysis to do descriptive statistics to understand you data. It is not required to show descriptive analysis here unless there is something important to report (transformations, deviation from normality, outliers, etc).

1. The data in the table give the infant mortality rates per 1000 livebirths in the United States for the period 1960-2000


![title](HW1.png)

a. Fit a linear regression line relating infant mortality rate to chronological year using these data. Use a data transformation if necessary

b. Test for significance of the linear relationship

c. If the present trend continues for the next 5 years what would be the predicted infant mortality rate in 2010


$2.$ The file HackerRank-Developer-Survey-2018-Numeric.csv shows an extensive survey obtained in 2016 to 25000 Hackers. The survey asked developers many questions around their skills, educational background, current role, and more... Answer the following questions using the data from this file. The file HackerRank-Developer-Survey-2018-Numeric-Mapping.csv has the metadata that explains the coding on each variable. 
This data was obtained from Kaggle at [https://www.kaggle.com/hackerrank/developer-survey-2018](https://www.kaggle.com/hackerrank/developer-survey-2018) There is more information regarding the survey and the data itself at the website.

a. Is there a relationship between gender and the Age that the hacker begin coding(q1AgeBeginCoding)? (Use a logistic regression)

b. Does age to begin coding, and gender have an effect on job level (q8JobLevel)? (Use MLR)

c. There is an increase focus on understanding trends about women pursuing careers as developers are there any relationships in the data that could be useful advice to women that want to pursue a career as a developer? The zip file has many other variables that you can use for this analysis. 

$3.$ The data set below represents the expected recovery time (in weeks) for patients after a surgical procedure with different post-operational methods for recovery. Negative values represent a faster recovery time than expected. The control patients followed a rutine post-operational for this type of procedure.

Using the data set below compare the effect of three treatments on the expected time of recovery after surgery. (Use ANOVA)

In [3]:
Recovery = c(.53, .36, .20, -.37, -.60, -.64, -.68, -1.27, .73, .31, .03, -.29, -.56, -.96, -1.61,
         -.78, -.86, -1.35, -1.48, -1.52, -2.04, -2.83)
treatment = c(rep("control",8), rep("surgery",7), rep("acupunture",7))
data = data.frame(Recovery, treatment)
